# Library Imports

In [1]:
import os
import numpy as np
import pandas as pd
from scipy import io
# from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from imblearn.over_sampling import RandomOverSampler
from src.helper_functions import load_data

Using TensorFlow backend.
/Users/sean/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/sean/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/sean/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/sean/anaconda3/envs

In [ ]:
import keras
from keras.callbacks import Callback
from keras.layers import Dense, Activation, Dropout
keras.__version__

# Load Data

In [ ]:
x_tr, y_tr, x_te, y_te = load_data()

In [ ]:
type(y_tr)

# Choose A Target

In [ ]:
y_tr.columns

In [ ]:
# for target in y_tr.columns:
target = 'NR.AhR'
rows_tr = np.isfinite(y_tr[target]).values
rows_te = np.isfinite(y_te[target]).values
x,y = x_tr[rows_tr], y_tr[target][rows_tr]
print('Target:',target, 'Training set shape:', x.shape)

# Address Class Imbalance

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, stratify=y, test_size=0.2, random_state=42)
ros = RandomOverSampler(random_state=0)
x_resampled, y_resampled = ros.fit_sample(x_train,y_train)

# Build Neural Network

Following the desciption in section 2.2.4 of the [DeepTox article](https://www.frontiersin.org/articles/10.3389/fenvs.2015.00080/full), I tried to use intermediate values in [Table 2](https://www.frontiersin.org/articles/10.3389/fenvs.2015.00080/full#T2) to build the neural network:

Following [this question/answer](https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model) and [this question/answer](https://stackoverflow.com/questions/54065733/how-to-employ-the-scikit-learn-evaluation-metrics-functions-with-keras-in-python) to implement usage of recall in model training:

In [ ]:
from src.metrics import recall_m, f1

In [ ]:
drop_out=0.5    # DeepTox range: 0.5, 0.2, 0
L2_reg = 0.0001 # Default = 0.01
layers = 3      # DeepTox range: 1, 2, 3, 4
act = 'sigmoid' # Consider sigmoid and tanh
neurons = 1024  # DeepTox range: 1024, 2048, 4096, 8192, 16384
# Info on decay: https://datascience.stackexchange.com/questions/26112/decay-parameter-in-keras-optimizers
decay = 0       # DeepTox range: 10^-4, 10^-5, 10^-6
learn_rate = 0.1  #Research appropriate range
DNN = keras.Sequential([
    keras.layers.InputLayer(input_shape=x.shape[1:],name='Input_Layer')
])
for i in range(1,layers+1):
    DNN.add(Dense(units=neurons, activation=act,\
                  name='h'+str(i)+'_'+act+'_activation',\
                  kernel_regularizer=keras.regularizers.l2(L2_reg)))
    DNN.add(Dropout(rate=drop_out,name='Dropout'+str(i)))
DNN.add(Dense(units=1, activation='sigmoid'))
keras.optimizers.Adam(lr=learn_rate, beta_1=0.9,\
                      beta_2=0.999, decay=decay, amsgrad=False)
DNN.compile(optimizer='adam', loss='binary_crossentropy',\
            metrics=['accuracy',recall_m,f1])
DNN.summary()

In [ ]:
DNN.fit(
    x_resampled, y_resampled, batch_size=512, epochs=100,\
    validation_data=(x_val,y_val), verbose=1,
    callbacks=[
        keras.callbacks.EarlyStopping(patience=16,verbose=1,\
                                      restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=3,verbose=1)
    ])

In [ ]:
auc_te = roc_auc_score(y_te[target][rows_te], DNN.predict(x_te[rows_te]))
print("%15s: %3.5f" % (target, auc_te))

# Precision-Recall Curve
roc_auc can be misleading when imbalanced data:<sup>[1](https://stats.stackexchange.com/questions/90779/area-under-the-roc-curve-or-area-under-the-pr-curve-for-imbalanced-data),[2](https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/),[3](https://scikit-learn.org/stable/modules/model_evaluation.html#precision-recall-f-measure-metrics)</sup>
>However, when dealing with highly skewed datasets, Precision-Recall (PR) curves give a more informative picture of an algorithm's performance. - Davis and Goadrich

Use average precision from the precision-recall curve for a single value summary.

In [ ]:
# Precision-Recall Curve 
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve

In [ ]:
average_precision=average_precision_score(y_te[target][rows_te],DNN.predict(x_te[rows_te]))
average_precision

Combining code from [here](https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/) and [here](https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html#sphx-glr-auto-examples-model-selection-plot-precision-recall-py)

In [ ]:
import matplotlib.pyplot as plt
from inspect import signature

testy=y_te[target][rows_te]
precision, recall, _ = precision_recall_curve(testy,DNN.predict(x_te[rows_te]))

# In matplotlib < 1.5, plt.fill_between does not have a 'step' argument
step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', label='DNN', **step_kwargs)

no_skill = len(testy[testy==1]) / len(testy)
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.legend()
plt.title(target+' Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision));

In [ ]:
y_testing=y_te[target][~np.isnan(y_te[target])]
y_hat_testing=DNN.predict_classes(x_te[rows_te])
print(np.array([['TN','FP'],['FN','TP']]))
print(confusion_matrix(y_testing,y_hat_testing))

In [ ]:
print('f1:',f1_score(y_testing,y_hat_testing))
print('recall:',recall_score(y_testing,y_hat_testing))
print('precision:',precision_score(y_testing,y_hat_testing))

In [ ]:
y_te[target][rows_te].value_counts()

In [ ]:
537/(537+73)

Uncomment to save model.  Last ROC_UAC = 0.86068

In [ ]:
# DNN.save('./models/saves/first_model.h5')